In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
layer_spec = [
    {"type": "conv", "shape": (3,3,1,10)},
    {"type": "conv", "shape": (3,3,10,10)},
    {"type": "pool", "shape": (2,2)},
    {"type": "conv", "shape": (3,3,10,20)},
    {"type": "pool", "shape": (2,2)},
    {"type": "conv", "shape": (3,3,20,20)},
    {"type": "flat", "shape": 180},
    {"type": "full", "shape": (180,40)},
    {"type": "logr", "shape": (40,10)}
]

In [ ]:
tf.reset_default_graph()
x = tf.placeholder('float32', shape=[None,28,28,1], name='x')
inp = x
for l,layer in enumerate(layer_spec):
    ltype = layer["type"]
    lshape = layer["shape"]
    
    if ltype == "conv":
        with tf.name_scope('conv{}'.format(l)):
            k = tf.Variable(
                tf.truncated_normal(shape=lshape, mean=0.0, stddev=0.05),
                name="kernel"
            )
            b = tf.Variable(tf.zeros((lshape[-1],)), name="bias")
            convout = tf.nn.conv2d(inp,k,strides=(1,1,1,1),
                                   padding="VALID")
            p = tf.nn.bias_add(convout, b)
            h = tf.nn.relu(p)
            inp = h
            
    elif ltype == "pool":
        with tf.name_scope('pool{}'.format(l)):
            out = tf.nn.max_pool(inp, ksize=(1,)+lshape+(1,), 
                                strides=(1,)+lshape+(1,), 
                                padding="VALID")
            inp = out
            
    elif ltype == "flat":
        with tf.name_scope('flat{}'.format(l)):
            out = tf.reshape(inp, (-1,lshape))
            inp = out
    
    elif ltype == "full":
        with tf.name_scope('full{}'.format(l)):
            w = tf.Variable(
                tf.truncated_normal(
                    shape=lshape,
                    mean=0.0, stddev=0.05),
                name='weight'
            )
            b = tf.Variable(tf.zeros(lshape[-1],), name="bias")
            p = tf.nn.bias_add(tf.matmul(inp, w), b)
            h = tf.nn.relu(p)
            inp = h
    elif ltype == "logr":
        with tf.name_scope('logr{}'.format(l)):
            assert l == len(layer_spec)-1
            w = tf.Variable(
                tf.truncated_normal(
                    shape=lshape,
                    mean=0.0, stddev=0.05),
                name='weight'
            )
            b = tf.Variable(tf.zeros(lshape[-1],), name="bias")
            p = tf.nn.bias_add(tf.matmul(inp, w), b)
            final_out = p

            
            
t = tf.placeholder('int32', name='t')
xent = tf.nn.sparse_softmax_cross_entropy_with_logits(
            final_out, t, name='xent')

loss = tf.reduce_mean(xent, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

#pred = tf.argmax(final_out, axis=1)
pred = tf.reduce_max(final_out, reduction_indices=[1])
correct = tf.nn.in_top_k(final_out, t, 1)
acc = tf.reduce_mean(tf.cast(correct, tf.float32))